In [ ]:
# 필요한 라이브러리 임포트
import cv2  # OpenCV, 이미지 처리를 위한 라이브러리
import numpy as np  # 배열 연산을 위한 라이브러리
import os  # 파일 및 디렉토리 작업을 위한 모듈
import easyocr  # OCR(광학 문자 인식)을 위한 라이브러리

# 설정 변수 정의
UPLOAD_FOLDER = "C:/workspace_project/AIproject/workspace/recieved_images"  # 이미지를 저장할 폴더 경로
MODEL_STORAGE_DIRECTORY = "C:/workspace_project/AIproject/workspace/user_network_dir"  # OCR 모델 저장 경로
BLOCKED_CHARACTERS = "<>|]\:[+=`@#$%^&*'"  # OCR에서 제외할 특수 문자 목록

# 업로드 및 모델 폴더가 존재하는지 확인하고 없으면 생성
os.makedirs(UPLOAD_FOLDER, exist_ok=True)  # 업로드 폴더 생성
os.makedirs(MODEL_STORAGE_DIRECTORY, exist_ok=True)  # 모델 저장 폴더 생성

# 이미지 전처리 함수
def preprocess_image(image_path):
    image = cv2.imread(image_path)  # 이미지 파일 읽기
    # blurred_image = cv2.GaussianBlur(image, (5, 5), 0)  # 가우시안 블러로 노이즈 제거 (주석 처리됨)
    # denoised_image = cv2.medianBlur(blurred_image, 5)  # 미디언 블러로 추가 노이즈 제거 (주석 처리됨)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 컬러 이미지를 그레이스케일로 변환
    # equalized_image = cv2.equalizeHist(gray_image)  # 히스토그램 평활화로 대비 조정 (주석 처리됨)
    # adaptive_threshold_image = cv2.adaptiveThreshold(  # 적응형 임계값 처리 (주석 처리됨)
    #     equalized_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    # )
    return gray_image  # 전처리된 그레이스케일 이미지 반환

# 이미지에서 텍스트를 추출하는 함수
def extract_text_from_image(image_path, filename, languages=["ko"]):
    model_storage_directory = os.path.abspath(MODEL_STORAGE_DIRECTORY)  # 모델 저장 폴더의 절대 경로
    user_network_directory = os.path.abspath(MODEL_STORAGE_DIRECTORY)  # 사용자 네트워크 폴더의 절대 경로

    # 커스텀 모델을 사용한 OCR 리더 설정
    reader_custom = easyocr.Reader(
        lang_list=languages,  # 인식할 언어 목록 (기본값: 한국어)
        gpu=True,  # GPU 사용 여부
        model_storage_directory=model_storage_directory,  # 모델 저장 경로
        user_network_directory=user_network_directory,  # 사용자 네트워크 경로
        recog_network="custom"  # 커스텀 인식 네트워크 사용
    )
    # 기본 모델을 사용한 OCR 리더 설정
    reader_default = easyocr.Reader(lang_list=languages, gpu=True)  # 기본 한국어 설정

    preprocessed_image = preprocess_image(image_path)  # 이미지 전처리
    result_custom = reader_custom.readtext(preprocessed_image, blocklist=BLOCKED_CHARACTERS)  # 커스텀 모델로 텍스트 인식
    result_default = reader_default.readtext(preprocessed_image, blocklist=BLOCKED_CHARACTERS)  # 기본 모델로 텍스트 인식

    # 커스텀 모델과 기본 모델의 결과를 비교하여 신뢰도가 높은 결과 선택
    result = []
    for custom, default in zip(result_custom, result_default):
        if custom[2] > default[2]:  # 커스텀 모델의 신뢰도가 더 높은 경우
            result.append(custom)
        else:
            result.append(default)

    extracted_data = []  # 추출된 텍스트와 신뢰도를 저장할 리스트
    loaded_image = cv2.imread(image_path)  # 원본 이미지 로드
    for bbox, text, confidence in result:
        extracted_data.append({"text": text, "confidence": confidence})  # 텍스트와 신뢰도 저장
        top_left = tuple(map(int, bbox[0]))  # 텍스트 영역의 좌상단 좌표
        bottom_right = tuple(map(int, bbox[2]))  # 텍스트 영역의 우하단 좌표
        cv2.rectangle(loaded_image, top_left, bottom_right, (255, 0, 0), 2)  # 텍스트 영역에 파란색 사각형 그리기
        text_position = (top_left[0], top_left[1] - 10)  # 신뢰도 표시 위치
        cv2.putText(
            loaded_image,
            f"{confidence*100:.2f}%",  # 신뢰도를 퍼센트로 표시
            text_position,
            cv2.FONT_HERSHEY_SIMPLEX,  # 글꼴 설정
            0.8,  # 글꼴 크기
            (255, 0, 0),  # 파란색 텍스트
            2,  # 텍스트 두께
        )

    # 결과 이미지를 저장
    result_image_filename = os.path.join(UPLOAD_FOLDER, filename)
    cv2.imwrite(result_image_filename, loaded_image)  # 처리된 이미지를 지정 경로에 저장

    return extracted_data, result_image_filename  # 추출된 데이터와 저장된 이미지 경로 반환

# 예제 실행 함수
def main():
    test_image_path = "C:/workspace_project/AIproject/workspace/demo_images/04.jpg"  # 테스트 이미지 경로
    filename = os.path.basename(test_image_path)  # 파일 이름 추출

    # 이미지에서 텍스트 추출 실행
    extracted_data, result_image_path = extract_text_from_image(
        test_image_path, filename
    )
    if extracted_data:  # 텍스트가 추출된 경우
        print("Extracted Texts:")  # 추출된 텍스트 출력 시작
        for data in extracted_data:
            print(f"{data['text']} ({data['confidence']*100:.2f}%)")  # 텍스트와 신뢰도 출력
        print(f"Annotated image saved at {result_image_path}")  # 저장된 이미지 경로 출력
    else:
        print("No text found in image.")  # 텍스트가 없는 경우 메시지 출력

# 스크립트 실행 진입점
if __name__ == "__main__":
    main()  # main 함수 실행
